In [ ]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
import os
from langchain.chat_models import init_chat_model

# -----------------------------
# Define Portfolio State
# -----------------------------
class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    total_inr: float


# -----------------------------
# Step 1: Currency Tool
# -----------------------------
@tool
def get_currency_value(currency_code: str) -> float:
    '''
    Return the current value of a currency against USD.
    :param currency_code: ISO currency code (e.g., "USD", "INR", "EUR")
    :return: current value of the currency against USD
    '''
    print(f"[TOOL CALL] get_currency_value called with: {currency_code}")  # 👈 debug log
    return {
        "USD": 1.0,
        "INR": 0.012,
        "EUR": 1.09,
    }.get(currency_code.upper(), 0.0)


# -----------------------------
# Step 2: Portfolio Calculations
# -----------------------------
def calc_total(state: PortfolioState) -> PortfolioState:
    state["total_usd"] = state["amount_usd"] * 1.08   # add 8% interest
    return state

def convert_to_inr(state: PortfolioState) -> PortfolioState:
    state["total_inr"] = state["total_usd"] * 85  # Assume 1 USD = 85 INR
    return state


# -----------------------------
# Step 3: Initialize LLM + Agent
# -----------------------------
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
os.environ["OPENAI_API_KEY"] = ""
# Initialize OpenAI GPT model
llm = init_chat_model(
    "gpt-3.5-turbo",
    model_provider="openai",
    temperature=0
)
tools = [get_currency_value]
graph = create_react_agent(llm, tools)


# -----------------------------
# Step 4: Run Workflow
# -----------------------------
# Start with an initial portfolio
portfolio: PortfolioState = {"amount_usd": 1000.0, "total_usd": 0.0, "total_inr": 0.0}

# Step A: Apply portfolio functions
portfolio = calc_total(portfolio)
portfolio = convert_to_inr(portfolio)

# Step B: Ask the agent something (optional)
state = graph.invoke({"messages": [{"role": "user", "content": "What is the value of INR in USD?"}]})

# -----------------------------
# Step 5: Print Results
# -----------------------------
print("Agent Response:", state["messages"][-1].content)
print("Portfolio State:", portfolio)


[TOOL CALL] get_currency_value called with: INR
Agent Response: The value of 1 INR in USD is 0.012.
Portfolio State: {'amount_usd': 1000.0, 'total_usd': 1080.0, 'total_inr': 91800.0}
